<a href="https://colab.research.google.com/github/pradyumna4998/Intro-to-ML/blob/main/HW7_Q2_Intro_to_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Introduction to Machine Learning

                                            Homework 7

Question 2

Name : V.pradyumna

Student ID : 801345963

In [1]:
!pip install ipython-autotime
%load_ext autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.0 MB/s eta 0:00:00
time: 350 µs (started: 2024-12-04 20:00:30 +00:00)


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, classification_report
import seaborn as sns

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)
np.random.seed(123)

time: 25.5 s (started: 2024-12-04 20:00:30 +00:00)


In [3]:
 # Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

time: 51.7 ms (started: 2024-12-04 20:00:55 +00:00)


In [4]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

time: 505 µs (started: 2024-12-04 20:00:55 +00:00)


In [5]:
training_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and standard(std)
images = torch.stack([img_t for img_t, _ in training_dataset], dim=3)
mean = images.view(3, -1).mean(dim=1)
std = images.view(3, -1).std(dim=1)

100%|██████████| 170M/170M [00:04<00:00, 34.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
time: 27.9 s (started: 2024-12-04 20:00:56 +00:00)


In [6]:
mean

tensor([0.4914, 0.4822, 0.4465])

time: 37.6 ms (started: 2024-12-04 20:01:23 +00:00)


In [7]:
std

tensor([0.2470, 0.2435, 0.2616])

time: 3.84 ms (started: 2024-12-04 20:01:23 +00:00)


In [8]:
cifar10 = datasets.CIFAR10(
    './data', train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2470, 0.2435, 0.2616))
    ]))

time: 474 ms (started: 2024-12-04 20:01:23 +00:00)


In [9]:
cifar10_val = datasets.CIFAR10(
     './data', train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2470, 0.2435, 0.2616))
    ]))

time: 358 ms (started: 2024-12-04 20:01:24 +00:00)


In [10]:
training_loader = DataLoader(cifar10, batch_size=32, shuffle=True, num_workers=2)
testing_loader = DataLoader(cifar10_val, batch_size=32, shuffle=False, num_workers=2)

time: 975 µs (started: 2024-12-04 20:01:24 +00:00)


In [11]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

time: 420 µs (started: 2024-12-04 20:01:24 +00:00)


In [12]:
print(torch.__version__)

2.5.1+cu121
time: 411 µs (started: 2024-12-04 20:01:24 +00:00)


In [13]:

import datetime
def training_loop(epochs, optimizer, model, loss_function, training_loader):
    for epoch in range(1, epochs + 1):
        training_loss = 0.0
        for images, labels in training_loader:
            images = images.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(images)
            loss = loss_function(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            training_loss += loss.item()

        if epoch == 1 or epoch % 2 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                training_loss / len(training_loader)))



time: 565 µs (started: 2024-12-04 20:01:24 +00:00)


In [14]:
class Residual_Block(nn.Module):
    def __init__(self, n_chans):
        super(Residual_Block, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)  # <1>
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  # <2>
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

time: 575 µs (started: 2024-12-04 20:01:24 +00:00)


In [15]:

import torch.nn.functional as F
class NetResDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [Residual_Block(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

time: 833 µs (started: 2024-12-04 20:01:24 +00:00)


In [16]:
device

device(type='cuda')

time: 3.4 ms (started: 2024-12-04 20:01:24 +00:00)


In [ ]:
model = NetResDeep(n_chans1=32, n_blocks=10).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=3e-3)
loss_fn = nn.CrossEntropyLoss()
training_loop(
    epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_function = loss_fn,
    training_loader = training_loader,
)

2024-12-04 20:01:59.241117 Epoch 1, Training loss 1.581621990627916
2024-12-04 20:02:31.600274 Epoch 2, Training loss 1.2795808721031048
2024-12-04 20:03:37.986840 Epoch 4, Training loss 1.0715397235604334
2024-12-04 20:04:42.162634 Epoch 6, Training loss 0.9441771852382848
2024-12-04 20:05:48.066993 Epoch 8, Training loss 0.8640034083021961
2024-12-04 20:06:52.118347 Epoch 10, Training loss 0.8036986036858952
2024-12-04 20:07:58.173224 Epoch 12, Training loss 0.752829780924877
2024-12-04 20:09:03.859696 Epoch 14, Training loss 0.7067300276979757
2024-12-04 20:10:08.526991 Epoch 16, Training loss 0.66741770820517
2024-12-04 20:11:14.616550 Epoch 18, Training loss 0.6336688590942098
2024-12-04 20:12:19.349393 Epoch 20, Training loss 0.5936948609367366
2024-12-04 20:13:25.009533 Epoch 22, Training loss 0.5680613198954557
2024-12-04 20:14:30.859550 Epoch 24, Training loss 0.5395997961362203
2024-12-04 20:15:35.072069 Epoch 26, Training loss 0.5091024690372625
2024-12-04 20:16:40.801647 Ep

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)
def validate(model, train_loader, val_loader):
    acc_dict = {}
    predictions = []
    exp_labels = []

    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <1>
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

                predictions.extend(predicted.cpu().numpy())
                exp_labels.extend(labels.cpu().numpy())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        acc_dict[name] = correct / total
    return acc_dict, predictions, exp_labels

In [ ]:
accuracy, predictions, expected_labels = validate(model, train_loader, val_loader)

In [ ]:
print(classification_report(predictions, expected_labels, target_names=class_names))

In [ ]:
precision = precision_score(predictions, expected_labels, average='macro')
recall = recall_score(predictions, expected_labels, average='macro')
cnf_matrix = confusion_matrix(predictions, expected_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_matrix, annot=True, cmap="viridis", fmt='g',
            xticklabels = class_names, yticklabels = class_names)
plt.title('Resnet10 Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45)
plt.yticks(rotation=45)

plt.show()

In [ ]:
class NetWidth(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out


In [ ]:
def training_loop_l2reg(n_epochs, optimizer, model, loss_fn,
                        train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            l2_lambda = 0.001
            l2_norm = sum(p.pow(2.0).sum()
                          for p in model.parameters())  # <1>
            loss = loss + l2_lambda * l2_norm

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()
        if epoch == 1 or epoch % 2 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))


In [ ]:
model_wd = NetWidth(n_chans1=32).to(device=device)
optimizer_wd = optim.SGD(model_wd.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()


training_loop_l2reg(
    n_epochs = 200,
    optimizer = optimizer_wd,
    model = model_wd,
    loss_fn = loss_fn,
    train_loader = training_loader,
)

In [ ]:

accuracy_weight_delay, predictions_weight_delay, expected_labels_weight_delay = validate(model_wd, train_loader, val_loader)


In [ ]:
print(classification_report(predictions_weight_delay, expected_labels_weight_delay, target_names=class_names))

In [ ]:
precision_weight_delay = precision_score(predictions_weight_delay, expected_labels_weight_delay, average='macro')
recall_weight_delay = recall_score(predictions_weight_delay, expected_labels_weight_delay, average='macro')
cnf_matrix_weight_delay = confusion_matrix(predictions_weight_delay, expected_labels_weight_delay)


In [ ]:

plt.figure(figsize=(8, 6))
sns.heatmap(cnf_matrix_weight_delay, annot=True, cmap="viridis", fmt='g',
            xticklabels = class_names, yticklabels = class_names)
plt.title('Resnet10 Confusion Matrix(Weight decay = 0.01)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45)
plt.yticks(rotation=45)

plt.show()


In [ ]:
class NetDropout(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.3)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv2_dropout = nn.Dropout2d(p=0.3)
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model_dropout = NetDropout(n_chans1=32).to(device=device)
optimizer_dropout = optim.SGD(model_dropout.parameters(), lr=1e-2)
loss_function = nn.CrossEntropyLoss()

training_loop(
    epochs = 300,
    optimizer = optimizer_dropout,
    model = model_dropout,
    loss_function = loss_function,
    training_loader = training_loader,
)


In [ ]:
accuracy_dropout, predictions_dropout, expected_labels_dropout = validate(model_dropout, train_loader, val_loader)

In [ ]:
print(classification_report(predictions_dropout, expected_labels_dropout, target_names=class_names))


In [ ]:
precision_dropout = precision_score(predictions_dropout, expected_labels_dropout, average='macro')
recall_dropout = recall_score(predictions_dropout, expected_labels_dropout, average='macro')
cnf_matrix_dropout = confusion_matrix(predictions_dropout, expected_labels_dropout)

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_matrix_dropout, annot=True, cmap="viridis", fmt='g',
            xticklabels = class_names, yticklabels = class_names)
plt.title('Resnet10  Confusion Matrix(dropout with p = 0.3)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45)
plt.yticks(rotation=45)

plt.show()


In [ ]:
class NetBatchNorm(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_batchnorm = nn.BatchNorm2d(num_features=n_chans1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv2_batchnorm = nn.BatchNorm2d(num_features=n_chans1 // 2)
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = self.conv1_batchnorm(self.conv1(x))
        out = F.max_pool2d(torch.tanh(out), 2)
        out = self.conv2_batchnorm(self.conv2(out))
        out = F.max_pool2d(torch.tanh(out), 2)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model_batch_normalization = NetBatchNorm(n_chans1=32).to(device=device)
optimizer_batch_normalization = optim.SGD(model_batch_normalization.parameters(), lr=1e-2)
loss_function = nn.CrossEntropyLoss()

training_loop(
    epochs = 300,
    optimizer = optimizer_batch_normalization,
    model = model_batch_normalization,
    loss_function = loss_function,
    training_loader = training_loader,
)

In [ ]:
accuracy_batch_normalization, predictions_batch_normalization, expected_labels_batch_normalization = validate(model_dropout, train_loader, val_loader)

In [ ]:
print(classification_report(predictions_dropout, expected_labels_dropout, target_names=class_names))

In [ ]:
precision_batch_norm = precision_score(predictions_batch_normalization, expected_labels_batch_normalization, average='macro')
recall_batch_norm = recall_score(predictions_batch_normalization, expected_labels_batch_normalization, average='macro')
cnf_matrix_batch_norm = confusion_matrix(predictions_batch_normalization, expected_labels_batch_normalization)

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_matrix_batch_norm, annot=True, cmap="viridis", fmt='g',
            xticklabels = class_names, yticklabels = class_names)
plt.title('Resnet10 Confusion Matrix(Batch Normalization)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45)
plt.yticks(rotation=45)

plt.show()
